In [106]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

import pandas as pd
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [107]:
train_data = datasets.CIFAR10(root='./data',
                              train=True,
                              transform=ToTensor(),
                              download=True
                            )

test_data = datasets.CIFAR10(root='./data',
                              train=False,
                              transform=ToTensor(),
                              download=True
                            )

Files already downloaded and verified
Files already downloaded and verified


In [108]:
BATCH_SIZE = 32

train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [109]:
test_img = next(iter(train_dataloader))[0][0].unsqueeze(dim=0)
test_img.shape

torch.Size([1, 3, 32, 32])

In [112]:
# Build the model

class ScratchNetwork(nn.Module):
    def __init__(self, input_channels, hidden_units, output_shape, kernel_size):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(input_channels, hidden_units, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*6*6, output_shape)
        )
        
    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x
    
model = ScratchNetwork(3, 64, 10, 3)

In [113]:
model(test_img)

tensor([[ 0.0225, -0.0258, -0.0464,  0.0566, -0.0278, -0.0526, -0.0543, -0.1302,
          0.0007, -0.1489]], grad_fn=<AddmmBackward0>)

In [117]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [139]:
from sklearn.metrics import accuracy_score

def train_step(model,
               dataloader,
               optimizer,
               loss_fn,
               device
              ):
    
    model.train()
    
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        
        # forward pass
        logits = model(X)
        y_pred = torch.softmax(logits, dim=1).argmax(dim=1)
        
        # calculate loss
        loss = loss_fn(logits, y)
        acc = accuracy_score(y_pred, y)
        train_loss += loss.item()
        train_acc += acc
        
        # zero Grad
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    train_loss = train_loss / len(dataloader) 
    train_acc = train_acc / len(dataloader)
    
    return train_loss, train_acc
        
def test_step(model,
              dataloader,
              optimizer,
              loss_fn,
              device
             ):
    
    model.eval()
    
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        
        for batch, (X, y) in enumerate(dataloader):
            
            logits = model(X)
            y_pred = torch.softmax(logits, dim=1).argmax(dim=1)
            
            loss = loss_fn(logits, y)
            acc = accuracy_score(y_pred, y)
            
            test_loss += loss.item()
            test_acc += acc
            
        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)
        
        return test_loss, test_acc
    

In [140]:
train_step(model, train_dataloader, optimizer, loss_fn, device)

(1.4354474426688708, 0.4925023992322457)

In [141]:
test_step(model, test_dataloader, optimizer, loss_fn, device)

(1.478673025822868, 0.47623801916932906)